# 01. Kigali Road Network Extraction
**Objective**: Extract a clean, drivable road network for the Kigali simulation area.

**Expected Outcome**: A validated SUMO `.net.xml` file and a filtered OpenStreetMap graph.

In [9]:
import sys
import os
from pathlib import Path

# Path Management
root_path = Path(os.getcwd()).parent
if str(root_path) not in sys.path:
    sys.path.append(str(root_path))

# Standard Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import osmnx as ox

# Quality of Life
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Task 1: Defining the Bounding Box
We will define the geographical boundaries for the simulation covering the primary sectors of Kigali (Nyarugenge, Gasabo, Kicukiro).

In [10]:
from src.utils.logger import setup_logger

# Initialize logger
logger = setup_logger("extraction_step")

# Test log
logger.info("Logger initialized successfully in notebook 01.")

2026-02-23 17:13:25 | INFO | extraction_step | Logger initialized successfully in notebook 01.
